In [1]:
# Lab 9 XOR
import tensorflow as tf
import numpy as np
import pandas as pd
tf.set_random_seed(777)  # for reproducibility

def min_max_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)


xy1=pd.read_csv('datasets_Wine.csv')
xy=xy1.values
for i in range(0,xy.shape[0]):
    if(xy[i][-1]=='good'):
        xy[i][-1]=1
    elif(xy[i][-1]=='bad'):
        xy[i][-1]=0
xy[:,:-1]=min_max_scaler(xy[:,:-1])
test_line=int(len(xy)*0.7)
x_data=xy[:test_line,:-1]
y_data=xy[:test_line,[-1]] 
x_test=xy[test_line:,:-1]
y_test=xy[test_line:,[-1]]
X = tf.placeholder(tf.float32, [None,x_data.shape[1]])
Y = tf.placeholder(tf.float32, [None, 1])

C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\USER\Anaconda3\envs\su\lib\site-packages\tensorflow\python\framework\dtypes.

In [2]:
W1 = tf.Variable(tf.random_normal([x_data.shape[1], 100]), name='weight1')
b1 = tf.Variable(tf.random_normal([100]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([100, 50]), name='weight2')
b2 = tf.Variable(tf.random_normal([50]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([50, 25]), name='weight2')
b3 = tf.Variable(tf.random_normal([25]), name='bias2')
layer3 = tf.sigmoid(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([25, 10]), name='weight2')
b4 = tf.Variable(tf.random_normal([10]), name='bias2')
layer4 = tf.sigmoid(tf.matmul(layer3, W4) + b4)

W5 = tf.Variable(tf.random_normal([10, 1]), name='weight4')
b5 = tf.Variable(tf.random_normal([1]), name='bias4')
hypothesis = tf.sigmoid(tf.matmul(layer4, W5) + b5)
save_file = './train_model_Wine_without_dropout.ckpt'
saver = tf.train.Saver()

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(5001):
        _, cost_val = sess.run([train, cost], feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            print(step, cost_val)
    
    # Accuracy report
    h, c, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_test, Y: y_test}
    )
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.9499935
100 0.64953697
200 0.61347455
300 0.5875427
400 0.5687715
500 0.55415195
600 0.5420388
700 0.53169435
800 0.5232079
900 0.5170774
1000 0.5128305
1100 0.50978535
1200 0.5074543
1300 0.5055486
1400 0.5039059
1500 0.5024363
1600 0.5010868
1700 0.49982247
1800 0.49861926
1900 0.49746177
2000 0.49634066
2100 0.49525127
2200 0.49419135
2300 0.49315834
2400 0.4921502
2500 0.49116388
2600 0.4901968
2700 0.4892462
2800 0.4883099
2900 0.48738584
3000 0.4864719
3100 0.48556733
3200 0.48467046
3300 0.4837803
3400 0.4828961
3500 0.48201743
3600 0.48114368
3700 0.48027495
3800 0.47941086
3900 0.47855145
4000 0.47769713
4100 0.47684777
4200 0.4760037
4300 0.47516567
4400 0.47433355
4500 0.473508
4600 0.47268957
4700 0.47187844
4800 0.471345
4900 0.47584802
5000 0.475895

Hypothesis:  [[0.87115216]
 [0.96586674]
 [0.9147642 ]
 [0.91009057]
 [0.9186897 ]
 [0.8094362 ]
 [0.9404077 ]
 [0.95135736]
 [0.7858947 ]
 [0.59105986]
 [0.8120663 ]
 [0.79817486]
 [0.12388563]
 [0.9717045 ]
 [0.93146527